In [4]:
from scripts import TimestampPreprocessing as tp
import pandas as pd

In [6]:
df = pd.DataFrame({'time_stamp':[112]})
df

,time_stamp
0,112


In [7]:
tp.transform_timestamp(df)

,year,sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_minute,cos_minute,0
index,,,,,,,,,,
0,0.0,0.5,0.866025,0.201299,0.97953,0.0,1.0,0.106293,0.994335,1.0
